In [1]:
import sys
import os

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)

import pandas as pd 
import numpy as np 
import sklearn
from dir import *
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset, Subset
from sklearn.model_selection import train_test_split
from VAE_model import *
from VAE_model_2 import *
from VAE_model_single import *
from training import *
from extras import *
import mantel
from Bio import Phylo
from Bio.Phylo.TreeConstruction import DistanceMatrix
from scipy.spatial.distance import squareform
from sklearn.metrics import pairwise_distances
from collections import defaultdict
from scipy.stats import pearsonr, spearmanr
import random
# from skbio.stats.distance import mantel
from sklearn.decomposition import PCA
import pingouin as pg
from sklearn.cluster import KMeans
plt.style.use('ggplot')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
large_data = pd.read_csv(TEN_K_DATASET, index_col=[0], header=[0])

In [3]:
phylogroup_data = pd.read_csv(TEN_K_DATASET_PHYLOGROUPS, index_col=[0], header=[0])

In [4]:
data_without_lineage = large_data.drop(index=['Lineage'])
merged_df = pd.merge(data_without_lineage.transpose(), phylogroup_data, how='inner', left_index=True, right_on='ID')
print("Dataframe shape: ", merged_df.shape)

Dataframe shape:  (1663, 55040)


In [5]:
merged_df

,group_5501,group_4783,group_4456,group_6055,group_2859,group_3832,group_4223,group_2401,group_1236,group_3245,...,group_2066_1,mukF_1,group_955_2,group_1382_5,group_2149_7,group_2067_4,group_2068_8,group_2069_3,group_2071_4,Phylogroup
ID,,,,,,,,,,,,,,,,,,,,,
AIAW00000000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
AIBY00000000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
AIFN00000000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
NC_002655,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
NC_002695,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JHRE00000000,0,0,0,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,B1
JHRP00000000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,B1
JHRY00000000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,B1


In [6]:
all_genes = np.array(merged_df.columns[:-1])

In [7]:
merged_df.filter(regex='group').columns

Index(['group_5501', 'group_4783', 'group_4456', 'group_6055', 'group_2859',
       'group_3832', 'group_4223', 'group_2401', 'group_1236', 'group_3245',
       ...
       'group_1609_5', 'group_2066_1', 'group_955_2', 'group_1382_5',
       'group_2149_7', 'group_2067_4', 'group_2068_8', 'group_2069_3',
       'group_2071_4', 'Phylogroup'],
      dtype='object', length=45534)

In [8]:
merged_df.columns

Index(['group_5501', 'group_4783', 'group_4456', 'group_6055', 'group_2859',
       'group_3832', 'group_4223', 'group_2401', 'group_1236', 'group_3245',
       ...
       'group_2066_1', 'mukF_1', 'group_955_2', 'group_1382_5', 'group_2149_7',
       'group_2067_4', 'group_2068_8', 'group_2069_3', 'group_2071_4',
       'Phylogroup'],
      dtype='object', length=55040)

In [9]:
genes_with_names = np.array(merged_df.columns.symmetric_difference(merged_df.filter(regex='group').columns))

In [10]:
# import requests
# import concurrent
# from concurrent.futures import ThreadPoolExecutor, as_completed

In [11]:
# import requests

# def get_ncbi_geneid(gene_name):
#     url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=gene&term={gene_name}[Gene%20Name]"
#     response = requests.get(url)
#     gene_id = None
#     if response.status_code == 200:
#         from xml.etree import ElementTree as ET
#         root = ET.fromstring(response.content)
#         id_list = root.find('IdList')
#         if id_list is not None and len(id_list):
#             gene_id = id_list[0].text
#     return gene_id


In [12]:
# def get_kegg_id(ncbi_geneid):
#     url = f"https://rest.kegg.jp/conv/eco/ncbi-geneid:{ncbi_geneid}"
#     response = requests.get(url)
#     kegg_id = None
#     if response.status_code == 200:
#         lines = response.text.strip().split("\n")
#         if lines and "\t" in lines[0]:
#             kegg_id = lines[0].split("\t")[1]
#     return kegg_id

In [13]:
# def get_ko_terms(kegg_gene_id):
#     url = f"https://rest.kegg.jp/link/ko/{kegg_gene_id}"
#     response = requests.get(url)
#     ko_terms = []
#     if response.status_code == 200:
#         lines = response.text.strip().split("\n")
#         for line in lines:
#             parts = line.split("\t")
#             if len(parts) == 2:
#                 ko_terms.append(parts[1])
#     return ko_terms


In [14]:
# # Example usage
# gene_names = ['aaaT', 'aaaT_1', 'aaaT_1_1', 'zur', 'zur_2', 'zwf']  
# gene_ids = {gene: get_ncbi_geneid(gene) for gene in gene_names}
# kegg_ids = {gene: get_kegg_id(gene_id) for gene, gene_id in gene_ids.items() if gene_id}
# ko_terms = {gene: get_ko_terms(kegg_gene_id) for gene, kegg_gene_id in kegg_ids.items() if kegg_gene_id}


# print("Gene Names to NCBI GeneIDs:", gene_ids)
# print("NCBI GeneIDs to KEGG IDs:", kegg_ids)
# print("KEGG Gene IDs to KO terms:", ko_terms)

In [15]:
# for i in genes_with_names:
#     print(i)

In [16]:
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     # Step 1: Get NCBI GeneIDs
#     gene_ids_future = {gene: executor.submit(get_ncbi_geneid, gene) for gene in genes_with_names}
#     gene_ids = {gene: future.result() for gene, future in gene_ids_future.items()}

In [17]:
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     # Step 2: Get KEGG IDs
#     kegg_ids_future = {gene: executor.submit(get_kegg_id, gene_id) for gene, gene_id in gene_ids.items() if gene_id}
#     kegg_ids = {gene: future.result() for gene, future in kegg_ids_future.items()}

In [18]:
# with concurrent.futures.ThreadPoolExecutor() as executor:   
#     # Step 3: Get KO terms
#     ko_terms_future = {gene: executor.submit(get_ko_terms, kegg_gene_id) for gene, kegg_gene_id in kegg_ids.items() if kegg_gene_id}
#     ko_terms = {gene: future.result() for gene, future in ko_terms_future.items()}

In [19]:
# print("Gene Names to NCBI GeneIDs:", gene_ids)
# print("NCBI GeneIDs to KEGG IDs:", kegg_ids)
# print("KEGG Gene IDs to KO terms:", ko_terms)

In [20]:
# len([k for k,v in gene_ids.items() if v != None])

In [21]:
# len([k for k,v in kegg_ids.items() if v != None])

In [22]:
# len([k for k,v in ko_terms.items() if v != ''])

In [23]:
essential_genes = pd.read_csv('/Users/anastasiiashcherbakova/Desktop/mbo001183726st1.csv')

In [24]:
essential_genes

,Table S1. Essentiality classification for genes of the TL data,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Gene,Insertion Index Score,Log Likelihood Ratio,Essential,Non-essential,Unclear
1,thrL,0.393939394,31.43105904,FALSE,TRUE,FALSE
2,thrA,0.219244823,17.2007822,FALSE,TRUE,FALSE
3,thrB,0.265809218,21.23454858,FALSE,TRUE,FALSE
4,thrC,0.205905206,15.99492449,FALSE,TRUE,FALSE
...,...,...,...,...,...,...
4309,creC,0.220350877,17.29962361,FALSE,TRUE,FALSE
4310,creD,0.22172949,17.42258883,FALSE,TRUE,FALSE
4311,arcA,0.09762901,4.641237681,FALSE,TRUE,FALSE
4312,yjjY,0.219858156,17.25561278,FALSE,TRUE,FALSE


In [25]:
essential_genes.columns

Index(['Table S1. Essentiality classification for genes of the TL data',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5'],
      dtype='object')

In [26]:
essential_genes['Table S1. Essentiality classification for genes of the TL data']

0       Gene
1       thrL
2       thrA
3       thrB
4       thrC
        ... 
4309    creC
4310    creD
4311    arcA
4312    yjjY
4313    yjtD
Name: Table S1. Essentiality classification for genes of the TL data, Length: 4314, dtype: object

In [27]:
essentail_names_array = np.array(essential_genes['Table S1. Essentiality classification for genes of the TL data'][essential_genes['Unnamed: 3'] == 'TRUE'])

In [28]:
essentail_names_array

array(['rpsT', 'ribF', 'ileS', 'lspA', 'ispH', 'dapB', 'folA', 'lptD',
       'ftsL', 'ftsI', 'murE', 'murF', 'mraY', 'murD', 'ftsW', 'murG',
       'murC', 'ftsQ', 'ftsA', 'ftsZ', 'lpxC', 'secA', 'coaE', 'aceF',
       'lpd', 'can', 'hemL', 'erpA', 'dapD', 'map', 'rpsB', 'tsf', 'pyrH',
       'frr', 'dxr', 'ispU', 'cdsA', 'bamA', 'lpxD', 'fabZ', 'lpxA',
       'lpxB', 'dnaE', 'accA', 'tilS', 'proS', 'ykfM', 'hemB', 'ribD',
       'ribE', 'nusB', 'thiL', 'dxs', 'ispA', 'dnaX', 'adk', 'hemH',
       'lpxH', 'cysS', 'folD', 'lipA', 'mrdB', 'mrdA', 'nadD', 'holA',
       'lptE', 'leuS', 'lnt', 'ybeY', 'glnS', 'fldA', 'sucA', 'sucB',
       'cydA', 'cydB', 'cydX', 'infA', 'cydC', 'cydD', 'lolA', 'serS',
       'rpsA', 'msbA', 'lpxK', 'ycaR', 'kdsB', 'mukF', 'mukE', 'mukB',
       'asnS', 'fabA', 'tusE', 'lpxL', 'murJ', 'rpmF', 'fabH', 'fabD',
       'fabG', 'acpP', 'tmk', 'holB', 'lolC', 'lolD', 'lolE', 'purB',
       'mnmA', 'ymfE', 'cohE', 'iraM', 'pth', 'prs', 'ispE', 'lolB',
       'he

In [29]:
header = 'gene'

In [30]:
np.savetxt("/Users/anastasiiashcherbakova/git_projects/masters_project/data/essential_genes.csv", essentail_names_array, delimiter = ",", header=header, fmt='%s')

In [31]:
count = 0
for i in essentail_names_array:
    if i in all_genes:
        count += 1

In [32]:
print(f"{count} present in the dataset out of {len(essentail_names_array)}")

316 present in the dataset out of 358
